In [1]:
# imports
import sys
import pandas as pd
import numpy as np
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import Data.data_provider as dp
import core as cal
from estimators.IR_RF_estimator import IR_RF
from sklearn.ensemble import RandomForestClassifier


In [2]:
# params
calib_methods = cal.calib_methods.copy()
metrics = ["acc", "brier"]

data_list = ["spambase", "climate", "QSAR", "bank", "climate", "parkinsons", "vertebral", "ionosphere", "diabetes", "breast", "blod"]
# data_list = ["spambase", "climate"]

params = {
    "runs": 5,
    "n_estimators": 100,
    "oob": False,
    "test_split": 0.3,
    "calib_split": 0.01
}

In [3]:
calib_results_dict = {}

for data_name in data_list:

    # Data
    X, y = dp.load_data(data_name, "../../")
    
    data_dict = {} # results for each data set will be saved in here.
    for seed in range(params["runs"]): # running the same dataset multiple times
        # split the data
        data = cal.split_train_calib_test(data_name, X, y, params["test_split"], params["calib_split"], seed)
        models = {}
        # train model
        rf = RandomForestClassifier(n_estimators=params["n_estimators"], oob_score=params["oob"], random_state=seed).fit(data["x_train"], data["y_train"])
        models["RF_pet"] = rf

        # irrf_l = IR_RF(n_estimators=params["n_estimators"], oob_score=params["oob"], random_state=seed).fit(data["x_train"], data["y_train"])
        # models["RF_l"] = irrf_l

        irrf_ct = IR_RF(n_estimators=params["n_estimators"], oob_score=params["oob"], random_state=seed).fit(data["x_train"], data["y_train"])
        models["RF_ct"] = irrf_ct

        # calibration
        res = cal.model_calibration(models, data, metrics) # res is a dict with all the metrics results as well as RF probs and every calibration method decision for every test data point
        data_dict = cal.update_runs(data_dict, res) # calib results for every run for the same dataset is aggregated in data_dict (ex. acc of every run as an array)
    calib_results_dict.update(data_dict) # merge results of all datasets together

In [4]:
tables = cal.mean_and_ranking_table(calib_results_dict, metrics, models, data_list, mean_and_rank=True)

In [5]:
tables["brier"]

,RF_pet,RF_ct
Data,,
spambase,0.041178,0.041167
climate,0.080319,0.080319
QSAR,0.098597,0.098597
bank,0.007427,0.007427
climate,0.080319,0.080319
parkinsons,0.077569,0.077569
vertebral,0.107286,0.107286
ionosphere,0.058253,0.058253
diabetes,0.161532,0.161532


In [6]:
tables["acc"]

,RF_pet,RF_ct
Data,,
spambase,0.951629,0.951774
climate,0.885185,0.885185
QSAR,0.865615,0.865615
bank,0.990777,0.990777
climate,0.885185,0.885185
parkinsons,0.871186,0.871186
vertebral,0.845161,0.845161
ionosphere,0.933962,0.933962
diabetes,0.763636,0.763636
